In [3]:
import pandas as pd
import os
import re
from pyhanlp import *
from gensim import corpora, models, similarities
import json

In [4]:
for root, dirs, files in os.walk('./'): 
    for file in files:
        print(file)
        if '.xlsx' in file and '未完成' not in file:
            text = pd.read_excel(file)
            text.to_csv('LDA测试数据.csv', sep='|', mode='a', index=False)

教育培训.xlsx
旅游交通.xlsx
分词——统计.ipynb
电子电器.xlsx
.DS_Store
哈工大停用词表.txt
婚恋交友.xlsx
文本分类.ipynb
LDA主题模型.ipynb
房产家居.xlsx
分类模型测试.txt
物流快递.xlsx
未完成分类.xlsx
教育分词.txt
文本分类-checkpoint.ipynb
LDA主题模型-checkpoint.ipynb
分词——统计-checkpoint.ipynb


In [5]:
data = pd.read_csv('./LDA测试数据.csv', sep='|')

In [6]:
data.head()

,id,cpi_industrytext,cpi_industrytext2,cpi_brandtext,cpi_issue_text,cpi_consume_date,cpi_consume_amount,cpi_purpose,cpi_company_name,cpi_title,cpi_content
0,13363030,教育培训,教育培训,NaN,"投诉问题：退款问题,诱导消费,培训贷,虚假宣传,态度恶劣",2019-10-22,30000.0,投诉要求： 退款,四川仁和会计教育培训,诱导消费贷款，虚假宣传，假合同,本人在2017年通过家人缴费9800元到仁和会计学校高新校区学习会计核算班后，在2018年6...
1,13342681,教育培训,教育培训,NaN,投诉问题：欺诈学费学费,2019-10-16,16000.0,"投诉要求： 退款,要求一分不少退还学费",翰苑教育培训中心,起诉瀚苑教育机构,我在2017年报了一个教育机构，当时是在我大学学校门口的教育机构，当时报考的梁老师承诺我不用...
2,13391742,教育培训,教育培训,NaN,"投诉问题：服务不到位,虚假宣传",2019-10-31,36000.0,"投诉要求： 退款,作出处罚",惠州市突破教育培训中心,投诉惠州突破教育培训机构,我小孩是半路上转过来读书的，家里跟这边教材不一样。为了孩子能尽快适应这边学习，就去找培训机构...
3,12098077,教育培训,教育培训,NaN,投诉问题：导致报考再等一年,2019-09-07,10000.0,投诉要求： 赔偿,菏泽天虹教育培训学校,本来报考专升本结果给报成了高升本，导致报考再等一年，造成严重损失,本来*看到函授本科，自己是大专毕业工作后考虑到综合原因需要提升学历就找到了菏泽天虹教育培训学...
4,11025533,教育培训,教育培训,NaN,"投诉问题：客服不处理,服务不到位,虚假宣传,霸王条款",2019-07-22,2830.0,"投诉要求： 退款,改善服务",芸学教育培训,芸学教育培训,之前在抖音看到可以报考心理咨询师的广告，然后就填写了资料，后面一个叫芸学教育的谢老师打了我的...


In [7]:
#HanLP.Config.ShowTermNature = False
CoreStopWordDictionary = JClass("com.hankcs.hanlp.dictionary.stopword.CoreStopWordDictionary")

In [8]:
"""创建停用词列表"""
def stopwordslist():
    stopwords = [line.strip() for line in open('./哈工大停用词表.txt',encoding='UTF-8').readlines()]
    return stopwords

In [9]:
def seg_depart(sentence):
    sentence_depart = HanLP.segment(sentence)
    stopwords = stopwordslist()
    outstr = ''
    for word in sentence_depart:
        word = word.toString()
        judge = word.split('/')
        if judge[0] not in stopwords:
            outstr += judge[0]
            outstr += " "
    # outstr：'黄蜂 湖人 首发 科比 带伤 战 保罗 加索尔 ...'       
    return outstr

In [10]:
if not os.path.exists("./测试.txt"):
    outfilename = "./测试.txt"
    inputs = pd.read_csv('./LDA测试数据.csv', sep='|')
    outputs = open(outfilename, 'w', encoding='UTF-8')

    # 把非汉字的字符全部去掉
    for line in inputs['cpi_content']:
        line = re.sub(r'[^\u4e00-\u9fa5]+','',str(line))
        line_seg = seg_depart(line.strip())
        outputs.write(line_seg.strip() + '\n')

    outputs.close()
    print("删除停用词和分词成功！！！")

删除停用词和分词成功！！！


In [12]:
##pyLDAvis 主题模型可视化

In [11]:
"""准备好训练语料，整理成gensim需要的输入格式"""
fr = open('./测试.txt', 'r',encoding='utf-8')
train = []
for line in fr.readlines():
    line = [word.strip() for word in line.split(' ')]
    train.append(line)

In [14]:
"""构建词频矩阵，训练LDA模型"""
dictionary = corpora.Dictionary(train)
# corpus[0]: [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1),...]
# corpus是把每条新闻ID化后的结果，每个元素是新闻中的每个词语，在字典中的ID和频率
corpus = [dictionary.doc2bow(text) for text in train]

lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=6)
topic_list = lda.print_topics(6)
print("6个主题的单词分布为：\n")
for topic in topic_list:
    print(topic)

6个主题的单词分布为：

(0, '0.022*"损坏" + 0.019*"合同" + 0.012*"消费者" + 0.009*"不" + 0.009*"规定" + 0.008*"要求" + 0.008*"服务" + 0.007*"公司" + 0.007*"条款" + 0.006*"中"')
(1, '0.045*"说" + 0.029*"顺丰" + 0.024*"不" + 0.020*"都" + 0.017*"没有" + 0.016*"还" + 0.011*"没" + 0.010*"打电话" + 0.009*"现在" + 0.009*"去"')
(2, '0.039*"快递" + 0.020*"客服" + 0.013*"联系" + 0.013*"没有" + 0.012*"不" + 0.010*"员" + 0.009*"号" + 0.009*"电话" + 0.009*"月" + 0.009*"赔偿"')
(3, '0.018*"" + 0.014*"公司" + 0.014*"催收" + 0.013*"元" + 0.012*"骚扰" + 0.012*"还" + 0.011*"电话" + 0.010*"通讯录" + 0.009*"小区" + 0.009*"信息"')
(4, '0.026*"电脑" + 0.013*"手机" + 0.012*"年" + 0.011*"问题" + 0.010*"售后" + 0.010*"开机" + 0.008*"鱼" + 0.008*"使用" + 0.008*"显示器" + 0.008*"闲"')
(5, '0.044*"顺丰" + 0.032*"元" + 0.022*"月" + 0.019*"客服" + 0.016*"押金" + 0.016*"没有" + 0.014*"不" + 0.013*"号" + 0.011*"退款" + 0.011*"还"')


## 评论主题

In [15]:
"""抽取新闻的主题"""
"""
# 用来测试的三条新闻，分别为体育、娱乐和科技新闻    
file_test = "./cnews.test.txt"
news_test = open(file_test, 'r', encoding='UTF-8')
"""
news_test = data['cpi_content'].iloc[0:3]
test = []
# 处理成正确的输入格式       
for line in news_test:
    #line = line.split('\t')[1]
    line = re.sub(r'[^\u4e00-\u9fa5]+','',line)
    line_seg = seg_depart(line.strip())
    line_seg = [word.strip() for word in line_seg.split(' ')]
    test.append(line_seg)    
    
# 新闻ID化    
corpus_test = [dictionary.doc2bow(text) for text in test]
# 得到每条新闻的主题分布
topics_test = lda.get_document_topics(corpus_test)  
labels = ['教育','jiaoyu','jiaoyu']
for i in range(3):
    print('这条'+labels[i]+'新闻的主题分布为：\n')
    print(topics_test[i],'\n')

#fr.close()
#news_test.close()

这条教育新闻的主题分布为：

[(0, 0.5019122), (1, 0.26264608), (3, 0.18277147), (4, 0.010993369), (5, 0.041176952)] 

这条jiaoyu新闻的主题分布为：

[(1, 0.47817162), (3, 0.4220511), (5, 0.091068506)] 

这条jiaoyu新闻的主题分布为：

[(0, 0.106965385), (1, 0.8033512), (2, 0.077292606), (3, 0.011370993)] 

